# Gradient Boosting Trees From Scratch
I will being by looking at the example of a GB Tree for regression. I will use the data from the autotrader task that has already been split, cleaned and encoded with the aim of predicting the price of a car.  I will begin by using sklearn's decision tree algorithm as a base, rather than implementing my own tree algorithm.

In [100]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import pandas as pd

RANDOM_SEED = 1337

## Notes on Tree-based Algorithms

Gradient Boosted Trees is another ensemble method, similar to a Random Forest algorithm. This means several tree-like models are trained in parallel and are all used to contribute to a final prediction. 

### Random Forests

In a Random Forest (RF), the first step is to produce a bootstrapped dataset (sampling with replacement). We then build a decision tree using this bootstrapped dataset - crucially, we use a random subset of the features at each step rather than the entire feature space. These 2 steps are repeated to produce a 'forest' of these random trees. 

**Random Forests on New Data**

When a new piece of data is passed through a RF, each tree provides an answer. In classification this could be yes/no, in regression this could be a continuous variable, like price. Once all the trees have given an answer, the final prediction is either the mode (in classification) or the mean (in regression) - this technique is called *bagging*. An out-of-bag score can then be calculated - the out-of-bag dataset are the examples that were not selected in the bootstrapping stage and acts as a validation set like in Cross Validation. In classification, the proportion of out-of-bag samples that were *incorrectly* classified is known as the out-of-bag error 

### Regression Trees
The inital root for a regression tree is the threshold for a feature which minimises the sum of the squared residuals. Each leaf continues in this fashion until the tree can no longer be split. To prevent overfitting, the algorithm typically has a min_samples_split hyperparameter. This prevents a split with a single observation.

# <a href="https://jerryfriedman.su.domains/ftp/trebst.pdf">Gradient Boosted Trees</a>
## Regression
Gradient Boost (GB) starts with a single leaf for the inital guess - for a continuous variable, this will be the average of the target. From this inital guess, GB trees are constructed but their overall size (number of leaves) is limited - typically this is between 8 and 32. Each successive tree is built upon the previous trees errors and once the maximum number of trees are constructed, a linear combination of these trees are utilised to make predictions. Each successive tree is scaled by the same amount - a constant usually defined as the learning rate or $\eta$.

 **AdaBoost** is a similar boosted tree algorithm - in this, however, the trees that are constructed are actually 'stumps' (this means a root and a single left and right leaf). Not only that but each successive stump is individually scaled based on how well it performed, rather than scalling all trees by a fixed $\eta$



In [101]:
# Creating some Dummy Data - from StatsQuest Video

# encoding: blue = 0, green = 1, red = 2
# encoding: male = 0 female = 1
df = pd.DataFrame({
    'height': [1.6, 1.6, 1.5, 1.8, 1.5, 1.4], 
    'fav_colour': [0, 1, 0, 2, 1, 0],
    'gender': [0, 1, 1, 0, 0, 1],
    'weight': [88, 76, 56, 73, 77, 57]
    })

df.head(10)

,height,fav_colour,gender,weight
0,1.6,0,0,88
1,1.6,1,1,76
2,1.5,0,1,56
3,1.8,2,0,73
4,1.5,1,0,77
5,1.4,0,1,57


In [102]:
y = df['weight'].reset_index()
X = df[['height', 'fav_colour', 'gender']]

#### Step 1: Create an initial guess
The initial guess or prediction in a regression GB tree is the mean of the target. For this simple example, the target is the weight of a person

In [103]:
initial_guess = y['weight'].mean()
print(f"Our inital guess is: {initial_guess}kg -> ({initial_guess:.2f}kg to 2 d.p)")

Our inital guess is: 71.16666666666667kg -> (71.17kg to 2 d.p)


### Step 2: Calculate a pseudo-residual
We can use our first prediction to calcuate our initial pseudo-residual. This can be calculated as (observed - predicted)

In [104]:
y['residual'] = y['weight'] - initial_guess
y.head(10)

,index,weight,residual
0,0,88,16.833333
1,1,76,4.833333
2,2,56,-15.166667
3,3,73,1.833333
4,4,77,5.833333
5,5,57,-14.166667


### Step 3: Build a tree to predict the residuals
Each successive tree is built on the errors (or residuals) from the previous tree. 

In [105]:
dt1 = DecisionTreeRegressor(random_state=RANDOM_SEED).fit(X, y['residual'])

### Step 4: Create new predictions based on the newly built tree(s)
Here I only have a single tree but typically, a large number of individual trees will be built and trained on successive residuals - the theory is that each successive tree adds small improvements to the overall prediction. The contribution of each new tree is controlled by a hyperparameter called the learning rate ($\eta$) - this is a constant scaling factor to prevent extreme overfitting.

AdaBoost, another popular gradient-boosted tree algorithm, has a dynamic 'learning rate' - this means each tree is scaled differently depending on how well the tree performed. 


Residuals are updated as follow:
$\textrm{y} - (\textrm{initial prediction} + (\textrm{learning rate} * \textrm{tree's prediction}))$

We can see below that with the creation of a new tree, our residuals have decreased by a small amount which means thats we have move a small step in the right direction. We can quantify this by calculating the sum of the residuals squared - we aim to minimise this as we build new trees


In [106]:
# Typically this is a small fraction (e.g. 0.1 or 0.3) but I have a single tree so I have increased it
learning_rate = 0.8
new_person = pd.DataFrame(
    {
    'height': [1.7], 
    'fav_colour': [1],
    'gender': [0],
    'weight': [75]
    }
)

y['new_residuals'] = y.weight - (initial_guess + (dt1.predict(X) * learning_rate))

print(f"Original sum of residuals squared : {sum(y.residual) ** 2}")
print(f"New sum of residuals squared : {sum(y.new_residuals) ** 2}")
y.head(10)

Original sum of residuals squared : 8.077935669463161e-28
New sum of residuals squared : 5.048709793414476e-29


,index,weight,residual,new_residuals
0,0,88,16.833333,3.366667
1,1,76,4.833333,0.966667
2,2,56,-15.166667,-3.033333
3,3,73,1.833333,0.366667
4,4,77,5.833333,1.166667
5,5,57,-14.166667,-2.833333


# Regression Full Example


In [107]:
# Load in data from autotrader data cleaner
X_train = pd.read_csv("data/X_train.csv").to_numpy()
X_test = pd.read_csv("data/X_test.csv").to_numpy()
y_train = pd.read_csv("data/y_train.csv")
y_test = pd.read_csv("data/y_test.csv")

print(f"Training data size: {len(X_train)}")
print(f"Testing data size: {len(X_test)}")

Training data size: 156170
Testing data size: 66930


In [108]:
NUM_OF_TREES = 20
ETA = 0.1

def objective(residuals):
    """Sum of Squared Residuals"""
    residual_sum = sum(residuals)
    return residual_sum ** 2


def train_gbt(X, y, num_of_trees, eta=0.1, target='price'):
    # TODO re-write without need for target
    y_copy = y.copy()

    # Step 1: Produce initial guess - regression GB tree initial is mean of target
    initial_guess = y_copy[target].mean()

    if 'residual' not in y_copy.columns:
        # Step 2: Produce initial pseudo-residual - this is calculated as observed - predicted
        y_copy['residual'] = y_copy[target] - initial_guess

        # Step 2.5: Calculate the sum of the residuals squared as a metric of training performance. We aim to minimise this
        ssr = objective(y_copy['residual'])
        print(f"Inital sum of squared residuals: {ssr}")

    # Initialise a list to keep track of trained trees    
    trees = []
    for _ in range(num_of_trees):
        # Limit max number of leaf nodes to 4 - this is predominantly for performance rather than overfitting
        dt = DecisionTreeRegressor(random_state=RANDOM_SEED, max_leaf_nodes=4) 

        # Step 3: Build a tree and fit it to the residuals (i.e. the error of the previous tree)
        dt.fit(X, y_copy['residual'])
        trees.append(dt) 

        # Initialise 'adjustment' - this is the factor by which successive trees influence the initial guess
        # This adjustment increases or decreases to get better predictions for each example
        tree_adjustment = 0
        for tree in trees:
            # Each tree's contribution to a prediction is scaled by a constant scalar called the learning rate
            # Each tree is scaled by same constant unlike AdaBoost which scales individual trees based on their performance
            tree_contribution = tree.predict(X) * learning_rate
            tree_adjustment += tree_contribution

        y_copy['residual'] = np.array(y_copy[target] - (initial_guess + tree_adjustment))
        

    print(f'Final SSR: {objective(y_copy.residual)}')
    # I return y_copy to see final residuals and a list of trained trees to feed to a predict function
    return y_copy, trees


def train_gbt_np(X, y):
    # TODO re-write without need for target
    residuals = y.copy().astype(float)

    # Step 1: Produce initial guess - regression GB tree initial is mean of target
    initial_guess = np.mean(residuals)

    # Step 2: Produce initial pseudo-residual - this is calculated as observed - predicted
    residuals -= initial_guess
    # Step 2.5: Calculate the sum of the residuals squared as a metric of training performance. We aim to minimise this
    ssr = objective(residuals)
    print(f"Inital sum of squared residuals: {ssr}")

    # Initialise a list to keep track of trained trees    
    trees = []
    for _ in range(NUM_OF_TREES):
        # Limit max number of leaf nodes to 4 - this is predominantly for performance rather than overfitting
        dt = DecisionTreeRegressor(random_state=RANDOM_SEED, max_leaf_nodes=4) 

        # Step 3: Build a tree and fit it to the residuals (i.e. the error of the previous tree)
        dt.fit(X, residuals)
        trees.append(dt)

        # Initialise 'adjustment' - this is the factor by which successive trees influence the initial guess
        # This adjustment increases or decreases to get better predictions for each example
        tree_adjustment = 0
        for tree in trees:
            # Each tree's contribution to a prediction is scaled by a constant scalar called the learning rate
            # Each tree is scaled by same constant unlike AdaBoost which scales individual trees based on their performance
            tree_contribution = tree.predict(X) * ETA
            tree_adjustment += tree_contribution

        residuals -= (initial_guess + tree_adjustment)
        
    print(f'Final SSR: {objective(residuals)}')
    return trees

In [109]:
new_y, tree_list = train_gbt(X_train, y_train, NUM_OF_TREES, ETA)
# tree_list = train_gbt_np(X_train, y_train)

Inital sum of squared residuals: 7.0909709531298234e-12
Final SSR: 3.143418974384534e-16


In [110]:
def gbt_predict(X_test, trees):
    # Reshape so we can broadcast
    y_preds = np.full((len(X_test), 1), fill_value=y_train.mean()).reshape(-1, )
    for tree in trees: 
        y_preds += learning_rate * tree.predict(X_test)
    return y_preds  
        
def mse(y_true, y_pred):
    """ Assuming both as np arrays """
    diff_squared = (y_true - y_pred) ** 2
    return diff_squared.mean()

def rmse(y_true, y_pred):
    return mse(y_true, y_pred) ** 0.5

def gbt_score(y_true, y_pred):
    y_true = y_true.to_numpy().reshape(-1, )
    print(f"RMSE: {rmse(y_true, y_pred):.2f}")

In [111]:
y_preds = gbt_predict(X_test, tree_list)
gbt_score(y_test, y_preds)

RMSE: 6825.83


# Classification

I have downloaded a simple **binary** classification dataset from Kaggle in which we try to predict whether someone has heart disease or not

In classification, the initial prediction for each example is the **log of the odds**. We can convert a log of the odds to a probability using the logit function seen below:

$\huge \frac{e^x}{1 + e^x}$

Where $x$ is the log of the odds

In [112]:
heart_df = pd.read_csv("data/heart.csv")

In [113]:
heart_X = heart_df.loc[:, heart_df.columns != 'output']
heart_y = heart_df.output

heart_X_train, heart_X_test, heart_y_train, heart_y_test = train_test_split(heart_X, heart_y)

In [114]:

ETA = 0.1
def get_log_odds(x, y):
    return np.log(x / y)

def logit(x):
    # Used to convert log of odds -> probability
    return np.exp(x) / (1 + np.exp(x))    

def score(y_true, y_pred):
    y_pred_label = [round(pred) for pred in y_pred]
    acc_score = accuracy_score(y_true, y_pred_label)
    f1 = f1_score(y_true, y_pred_label)

    return acc_score, f1

def train_gbt_classifier(X_train, y_train):
    """
    This only trains a single tree to imporve predictions because the dataset is relatively small
    """
    class_counts = Counter(y_train)
    util_df = pd.DataFrame()  # Will be used for storing residuals / probabilities

    initial_log_odds = get_log_odds(class_counts[1], class_counts[0])
    initial_probability = logit(initial_log_odds)
    initial_predictions = [round(initial_probability) for _ in range(len(y_train))]

    init_acc, init_f1 = score(y_train, initial_predictions)
    print(f"Initial Predictions\nAcc -> {init_acc:.2f}\nF1 -> {init_f1:.2f}")

    util_df['predicted_probability'] = [initial_probability for _ in range(len(y_train))]
    util_df['residual'] = np.array(y_train) - util_df['predicted_probability']

    dt = DecisionTreeRegressor(random_state=RANDOM_SEED, max_leaf_nodes=4).fit(X_train, util_df['residual'])

    util_df['leaf_index'] = dt.apply(X_train)
    util_df['one_minus_previous_probability'] = 1 - util_df['predicted_probability']

    leaf_map = dict()
    for ind in set(util_df['leaf_index']):
        sub_df = util_df.loc[util_df['leaf_index'] == ind].reset_index()
        residual_sum = sub_df['residual'].sum()
        sub_df['denominator'] = sub_df['predicted_probability'] * sub_df['one_minus_previous_probability']
        p_sum = sub_df['denominator'].sum()
        leaf_map[ind] = residual_sum / p_sum

    util_df['leaf_output'] = util_df.leaf_index.map(leaf_map) * ETA


    new_log_odd_preds = initial_log_odds + util_df['leaf_output']
    util_df['predicted_probability'] = logit(new_log_odd_preds)
    util_df['residual'] = y_train - util_df['predicted_probability']

    acc, f1 = score(y_train, util_df.predicted_probability)
    print(f"Final Predictions\nAcc -> {acc:.2f}\nF1 -> {f1:.2f}")

In [115]:
train_gbt_classifier(heart_X_train, heart_y_train)

Initial Predictions
Acc -> 0.54
F1 -> 0.70
Final Predictions
Acc -> 0.75
F1 -> 0.81
